In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2scLH1sEAgvMgcd1gXODceGVId4_2KLHR9jmVMVS1bD5VkWEi")

In [ ]:
!pip install flask-ngrok pyngrok scikit-surprise flask --quiet
!pip install flask pyngrok scikit-surprise --quiet

import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok

df = pd.read_csv("ratings.csv", nrows=100000)
df = df[["userId", "movieId", "rating"]]
df.columns = ["user_id", "movie_id", "rating"]

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

svd_model = SVD(n_factors=50)
svd_model.fit(trainset)
svd_predictions = svd_model.test(testset)
print("RMSE (SVD):", accuracy.rmse(svd_predictions))


knn_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
knn_model.fit(trainset)
knn_predictions = knn_model.test(testset)
print("RMSE (KNN):", accuracy.rmse(knn_predictions))


movies_df = pd.read_csv("movies.csv")
movies_df = movies_df[["movieId", "title"]]
movies_df.columns = ["movie_id", "title"]

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(movies_df["title"])
content_similarity = cosine_similarity(tfidf_matrix)


def hybrid_recommend(user_id, num_recommendations=5):
    all_movies = df["movie_id"].unique()
    rated_movies = df[df["user_id"] == user_id]["movie_id"].tolist()
    unrated_movies = [movie for movie in all_movies if movie not in rated_movies]


    svd_predictions = [svd_model.predict(user_id, movie) for movie in unrated_movies]
    sorted_svd = sorted(svd_predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]
    top_svd = [(pred.iid, pred.est) for pred in sorted_svd]


    content_scores = [(movie, content_similarity[movies_df[movies_df["movie_id"] == movie].index[0]].sum()) for movie in unrated_movies]
    sorted_content = sorted(content_scores, key=lambda x: x[1], reverse=True)[:num_recommendations]


    hybrid_results = list(set(top_svd + sorted_content))[:num_recommendations]

    movie_titles = []
    for movie_id, _ in hybrid_results:
        title = movies_df[movies_df["movie_id"] == movie_id]["title"].values[0]
        movie_titles.append(title)

    return movie_titles


app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string('''
     <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Movie Recommendation System</title>
    <style>
        /* Overall page background */
        body {
            font-family: Arial, sans-serif;
            background-color: #000000; /* Black background */
            color: #f4f4f9;
            margin: 0;
            padding: 0;
        }

        /* Heading style */
        h1 {
            background-color: #00FFFF; /* Aqua blue background */
            color: black;
            text-align: center;
            padding: 20px;
            margin: 0;
        }

        /* Centering form container */
        .container {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 80vh;
            padding: 20px;
        }

        /* Form styling */
        form {
            background-color: white;
            padding: 30px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            width: 100%;
            max-width: 400px;
        }

        /* Input field styling */
        input[type="number"] {
            width: 100%;
            padding: 12px;
            margin: 10px 0;
            border: 2px solid #ddd;
            border-radius: 4px;
            font-size: 16px;
        }

        /* Button styling */
        button {
            width: 100%;
            padding: 12px;
            background-color: #00FFFF; /* Aqua blue button */
            color: black;
            border: none;
            border-radius: 4px;
            font-size: 16px;
            cursor: pointer;
        }

        button:hover {
            background-color: #00b3b3; /* Darker aqua on hover */
        }

        /* Label text color changed to black */
        label {
            color: black;
            font-size: 18px;
            font-weight: bold;
        }

        /* Responsive design for smaller screens */
        @media (max-width: 600px) {
            h1 {
                font-size: 24px;
                padding: 15px;
            }

            form {
                padding: 20px;
            }

            input[type="number"], button {
                font-size: 14px;
            }
        }
    </style>
</head>
<body>

    <h1>Movie Recommendation System</h1>

    <div class="container">
        <form action="/recommend" method="get">
            <label for="user_id">Enter User ID:</label>
            <input type="number" id="user_id" name="user_id" required placeholder="Enter your user ID">
            <button type="submit">Get Recommendations</button>
        </form>
    </div>

</body>
</html>


    ''')


@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    recommendations = hybrid_recommend(user_id)


    return render_template_string('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Recommended Movies</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #000000; /* Black background */
            color: #f4f4f9;
            margin: 0;
            padding: 0;
        }

        h1 {
            background-color: #00FFFF; /* Aqua blue background */
            color: black;
            text-align: center;
            padding: 20px;
            margin: 0;
        }

        .container {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 80vh;
            padding: 20px;
        }

        ul {
            list-style-type: none;
            padding: 0;
            width: 100%;
            max-width: 600px;
        }

        li {
            background-color: #333;
            color: white;
            padding: 15px;
            margin: 10px 0;
            border-radius: 5px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            font-size: 18px;
        }

        a {
            color: #00FFFF;
            text-decoration: none;
            font-size: 18px;
            margin-top: 20px;
            display: inline-block;
        }

        a:hover {
            color: #00b3b3;
        }

        .back-button {
            margin-top: 20px;
            padding: 12px;
            background-color: #00FFFF;
            color: black;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            font-size: 16px;
        }

        .back-button:hover {
            background-color: #00b3b3;
        }

        /* Responsive design for smaller screens */
        @media (max-width: 600px) {
            h1 {
                font-size: 24px;
                padding: 15px;
            }

            ul {
                width: 90%;
            }

            li {
                font-size: 16px;
                padding: 10px;
            }

            a, .back-button {
                font-size: 14px;
            }
        }
    </style>
</head>
<body>

    <h1>Recommended Movies for User {{ user_id }}</h1>

    <div class="container">
        <ul>
            {% for movie in recommendations %}
                <li>{{ movie }}</li>
            {% endfor %}
        </ul>

        <a href="/" class="back-button">Go back</a>
    </div>

</body>
</html>

    ''', user_id=user_id, recommendations=recommendations)


public_url = ngrok.connect(5000).public_url
print(f" * Running on {public_url}")


app.run(port=5000)

RMSE: 0.9107
RMSE (SVD): 0.9106760680290868
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0097
RMSE (KNN): 1.0097036421415866
 * Running on https://8f1e-35-247-166-69.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:53:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:53:40] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:53:48] "GET /recommend?user_id=1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:53:50] "GET /recommend?user_id=1 HTTP/1.1" 200 -
